In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.append('/Users/ajc/CODES/ats/tools/utils/')
import get_transect_data

In [3]:
Data_B = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_B_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_B['temperature'] = np.array(Data)
Data_B['time'] = np.array(Time)

In [4]:
Data_S30 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S30_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S30['temperature'] = np.array(Data)
Data_S30['time'] = np.array(Time)

In [5]:
Data_S60 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S60_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S60['temperature'] = np.array(Data)
Data_S60['time'] = np.array(Time)

In [6]:
Data_S90 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S90_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S90['temperature'] = np.array(Data)
Data_S90['time'] = np.array(Time)

In [7]:
"""
    nvar, cycles, xnum, znum = data.shape
    RiparianT = dict()
    avgTemp = np.zeros(cycles)
    for i in range(cycles):
        avgTemp[i] = np.mean(data[nvar-1,i,0:10,55:])
    RiparianT['times'] = np.array(times)
    RiparianT['temp'] = np.array(avgTemp)
    for key in RiparianT.keys():
        RiparianT[key + '-annual_avg'] = np.reshape(RiparianT[key], (-1,365)).mean(axis=1)    
    return RiparianT
"""

"\n    nvar, cycles, xnum, znum = data.shape\n    RiparianT = dict()\n    avgTemp = np.zeros(cycles)\n    for i in range(cycles):\n        avgTemp[i] = np.mean(data[nvar-1,i,0:10,55:])\n    RiparianT['times'] = np.array(times)\n    RiparianT['temp'] = np.array(avgTemp)\n    for key in RiparianT.keys():\n        RiparianT[key + '-annual_avg'] = np.reshape(RiparianT[key], (-1,365)).mean(axis=1)    \n    return RiparianT\n"

In [8]:
def get_locationbased_ALT(season = " ",location=0):
    data = []
    times = []
    if season == "B":
        data = Data_B['temperature']
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30['temperature']
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60['temperature']
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90['temperature']
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT = dict()
    avgTemp = np.zeros(cycles)
    ThawTop = np.zeros(cycles)
    ThawBottom = np.zeros(cycles)
    
    ddepth = 0 
    for i in range(cycles):
        loc = np.where(data[nvar-1,i,location,:] > 273.15)[0]
        if len(loc) >0:
            #dz = -(80 - loc[0])*2.0
            dz =  (data[1,0,location,loc[0]-1] - data[1,0,location,loc[-1]])*100
            #print (loc)
            avgTemp[i] = dz
            ThawTop[i] = (5.015-data[1,0,location,loc[-1]])*100
            ThawBottom[i] = (5.015-data[1,0,location,loc[0]-1])*100
            #print (data[1,0,location,loc[-1]], loc[-1])
            #break
    
    HillslopeT['times'] = np.array(times)
    HillslopeT['temp'] = np.array(avgTemp)
    HillslopeT['temp_top'] = np.array(ThawTop)
    HillslopeT['temp_bottom'] = np.array(ThawBottom)
    L = len(HillslopeT['temp'])%365
    for key in HillslopeT.keys():
        HillslopeT[key + '-annual_avg'] = np.reshape(HillslopeT[key][:-L], (-1,365))   
    return HillslopeT

In [9]:
#HillslopeALT_B_loc10 = get_locationbased_ALT("B",location=Loc)

In [10]:
Loc = 0
HillslopeALT_B_loc10 = get_locationbased_ALT("B",location=Loc)
HillslopeALT_S30_loc10 = get_locationbased_ALT("S30",location=Loc)
HillslopeALT_S60_loc10 = get_locationbased_ALT("S60",location=Loc)
HillslopeALT_S90_loc10 = get_locationbased_ALT("S90",location=Loc)

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
#axs = axs.ravel()

#print HillslopeALT_B_loc10['times-annual_avg'].shape
thaw_layer = False
yr = 18
ax1=plt.subplot(111)
if thaw_layer == True:
    ax1.plot(HillslopeALT_B_loc10['times-annual_avg'][yr], HillslopeALT_B_loc10['temp-annual_avg'][yr],'k-',label='Basecase')
    ax1.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr], HillslopeALT_S30_loc10['temp-annual_avg'][yr],'g-',label=r'S$_{30}$')
    ax1.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr], HillslopeALT_S60_loc10['temp-annual_avg'][yr],'b-',label=r'S$_{60}$')
    ax1.plot(HillslopeALT_S90_loc10['times-annual_avg'][yr], HillslopeALT_S90_loc10['temp-annual_avg'][yr],'r-',label=r'S$_{90}$')
else:
    d_B = HillslopeALT_B_loc10['temp_top-annual_avg'][yr] - HillslopeALT_B_loc10['temp_bottom-annual_avg'][yr]
    mask = np.where(d_B==0)[0]
    data_B_top = np.delete(HillslopeALT_B_loc10['temp_top-annual_avg'][yr],mask)
    data_B_bottom = np.delete(HillslopeALT_B_loc10['temp_bottom-annual_avg'][yr],mask)
    time_B = np.delete(HillslopeALT_B_loc10['times-annual_avg'][yr],mask)
    ax1.plot(time_B, -data_B_top,marker='x',markersize='2',linestyle='none',color='k')#,label='Basecase')
    
    #-------------------
    d_S30 = HillslopeALT_S30_loc10['temp_top-annual_avg'][yr] - HillslopeALT_S30_loc10['temp_bottom-annual_avg'][yr]
    mask = np.where(d_S30==0)[0]
    data_S30_top = np.delete(HillslopeALT_S30_loc10['temp_top-annual_avg'][yr],mask)
    data_S30_bottom = np.delete(HillslopeALT_S30_loc10['temp_bottom-annual_avg'][yr],mask)
    time_S30 = np.delete(HillslopeALT_S30_loc10['times-annual_avg'][yr],mask)
    ax1.plot(time_S30, -data_S30_top,marker='x',markersize='2',linestyle='none',color='green')#,label='Basecase')

    #-------------------
    d_S60 = HillslopeALT_S60_loc10['temp_top-annual_avg'][yr] - HillslopeALT_S60_loc10['temp_bottom-annual_avg'][yr]
    mask = np.where(d_S60==0)[0]
    data_S60_top = np.delete(HillslopeALT_S60_loc10['temp_top-annual_avg'][yr],mask)
    data_S60_bottom = np.delete(HillslopeALT_S60_loc10['temp_bottom-annual_avg'][yr],mask)
    time_S60 = np.delete(HillslopeALT_S60_loc10['times-annual_avg'][yr],mask)
    ax1.plot(time_S60, -data_S60_top,marker='x',markersize='2',linestyle='none',color='blue')#,label='Basecase')

    #-------------------
    d_S90 = HillslopeALT_S90_loc10['temp_top-annual_avg'][yr] - HillslopeALT_S90_loc10['temp_bottom-annual_avg'][yr]
    mask = np.where(d_S90==0)[0]
    data_S90_top = np.delete(HillslopeALT_S90_loc10['temp_top-annual_avg'][yr],mask)
    data_S90_bottom = np.delete(HillslopeALT_S90_loc10['temp_bottom-annual_avg'][yr],mask)
    time_S90 = np.delete(HillslopeALT_S90_loc10['times-annual_avg'][yr],mask)
    ax1.plot(time_S90, -data_S90_top,marker='x',markersize='2',linestyle='none',color='red')#,label='Basecase')
    
    ax1.plot(time_B, -data_B_bottom,marker='x',markersize='2',linestyle='none',color='k',label='Basecase')
    ax1.plot(time_S30, -data_S30_bottom,marker='x',markersize='2',linestyle='none',color='green',label=r'S$_{30}$')
    ax1.plot(time_S60, -data_S60_bottom,marker='x',markersize='2',linestyle='none',color='blue',label=r'S$_{60}$')
    ax1.plot(time_S90, -data_S90_bottom,marker='x',markersize='2',linestyle='none',color='red',label=r'S$_{90}$')
    #ax1.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr], -HillslopeALT_S30_loc10['temp_bottom-annual_avg'][yr],'g-',label=r'S$_{30}$')
    #ax1.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr], -HillslopeALT_S60_loc10['temp_bottom-annual_avg'][yr],'b-',label=r'S$_{60}$')
    #ax1.plot(HillslopeALT_S90_loc10['times-annual_avg'][yr], -HillslopeALT_S90_loc10['temp_bottom-annual_avg'][yr],'r-',label=r'S$_{90}$')

print HillslopeALT_S90_loc10['times-annual_avg'][yr][0]%365
"""
yr = 18
ax2=plt.subplot(122)
ax2.plot(HillslopeALT_B_loc10['times-annual_avg'][yr], HillslopeALT_B_loc10['temp-annual_avg'][yr],'k-',label='Basecase')
ax2.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr], HillslopeALT_S30_loc10['temp-annual_avg'][yr],'g-',label=r'S$_{30}$')
ax2.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr], HillslopeALT_S60_loc10['temp-annual_avg'][yr],'b-',label=r'S$_{60}$')
ax2.plot(HillslopeALT_S90_loc10['times-annual_avg'][yr], HillslopeALT_S90_loc10['temp-annual_avg'][yr],'m-',label=r'S$_{90}$')
"""
ax1.set_title('Depth to base and top of the thaw layer (pond center)',fontsize='large', fontweight='normal')
ax1.set_xlabel('Time [Month of year]',fontsize='large', fontweight='normal')
ax1.set_ylabel('Depth [cm]',fontsize='large', fontweight='normal')
ax1.set_xlim(18.71,19.71)
ax1.set_xticklabels(['x', 'Oct', 'Dec', 'March', 'May', 'July'],fontsize=11, fontweight='normal')
ax1.set_yticklabels(np.linspace(90,0,10,dtype='i'),fontsize=11, fontweight='normal')
#ax1.set_ylim((-81, 1.5))
ax1.legend(loc='center left', fontsize=10,bbox_to_anchor=(.005,0.6, .1, .1))#, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
  
plt.tight_layout()
plt.savefig(outfile+'ALT-x%sm-5.pdf'%Loc, bbox_inches='tight', dpi=100)
plt.show()

In [49]:
from scipy.interpolate import interp1d
import scipy.signal

In [62]:
#smoothing December 20
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
#axs = axs.ravel()

#print HillslopeALT_B_loc10['times-annual_avg'].shape
thaw_layer = False
yr = 18
ax1=plt.subplot(111)
if thaw_layer == True:
    ax1.plot(HillslopeALT_B_loc10['times-annual_avg'][yr], HillslopeALT_B_loc10['temp-annual_avg'][yr],'k-',label='Basecase')
    ax1.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr], HillslopeALT_S30_loc10['temp-annual_avg'][yr],'g-',label=r'S$_{30}$')
    ax1.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr], HillslopeALT_S60_loc10['temp-annual_avg'][yr],'b-',label=r'S$_{60}$')
    ax1.plot(HillslopeALT_S90_loc10['times-annual_avg'][yr], HillslopeALT_S90_loc10['temp-annual_avg'][yr],'r-',label=r'S$_{90}$')
else:
    d_B = HillslopeALT_B_loc10['temp_top-annual_avg'][yr] - HillslopeALT_B_loc10['temp_bottom-annual_avg'][yr]
    mask = np.where(d_B==0)[0]
    print (mask)
    ax1.plot(HillslopeALT_B_loc10['times-annual_avg'][yr][:mask[0]], -HillslopeALT_B_loc10['temp_top-annual_avg'][yr][:mask[0]],linestyle='dashed',color='k',label='Basecase (t)')
    ax1.plot(HillslopeALT_B_loc10['times-annual_avg'][yr][mask[-1]:], -HillslopeALT_B_loc10['temp_top-annual_avg'][yr][mask[-1]:],linestyle='dashed',color='k')
    ax1.plot(HillslopeALT_B_loc10['times-annual_avg'][yr][:mask[0]], -HillslopeALT_B_loc10['temp_bottom-annual_avg'][yr][:mask[0]],linestyle='-',color='k',label='Basecase (b)')
    ax1.plot(HillslopeALT_B_loc10['times-annual_avg'][yr][mask[-1]:], -HillslopeALT_B_loc10['temp_bottom-annual_avg'][yr][mask[-1]:],linestyle='-',color='k')
    #f = scipy.interpolate.CubicSpline(HillslopeALT_B_loc10['times-annual_avg'][yr][:mask[0]], -HillslopeALT_B_loc10['temp_top-annual_avg'][yr][:mask[0]],bc_type='clamped')
    #xnew = np.linspace(HillslopeALT_B_loc10['times-annual_avg'][yr][0],HillslopeALT_B_loc10['times-annual_avg'][yr][mask[0]-1],1000)
    #ax1.plot(xnew, f(xnew),linestyle='dashed',color='r',label='Basecase1')
    
    #-------------------
    d_S30 = HillslopeALT_S30_loc10['temp_top-annual_avg'][yr] - HillslopeALT_S30_loc10['temp_bottom-annual_avg'][yr]
    mask = np.where(d_S30==0)[0]
    ax1.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr][:mask[0]], -HillslopeALT_S30_loc10['temp_top-annual_avg'][yr][:mask[0]],linestyle='dashed',color='g',label=r'S$_{30}$ (t)')
    ax1.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr][mask[-1]:], -HillslopeALT_S30_loc10['temp_top-annual_avg'][yr][mask[-1]:],linestyle='dashed',color='g')
    ax1.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr][:mask[0]], -HillslopeALT_S30_loc10['temp_bottom-annual_avg'][yr][:mask[0]],linestyle='-',color='g',label=r'S$_{30}$ (b)')
    ax1.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr][mask[-1]:], -HillslopeALT_S30_loc10['temp_bottom-annual_avg'][yr][mask[-1]:],linestyle='-',color='g')
    
    #-------------------
    d_S60 = HillslopeALT_S60_loc10['temp_top-annual_avg'][yr] - HillslopeALT_S60_loc10['temp_bottom-annual_avg'][yr]
    mask = np.where(d_S60==0)[0]
    ax1.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr][:mask[0]], -HillslopeALT_S60_loc10['temp_top-annual_avg'][yr][:mask[0]],linestyle='dashed',color='b',label=r'S$_{60}$ (t)')
    ax1.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr][mask[-1]:], -HillslopeALT_S60_loc10['temp_top-annual_avg'][yr][mask[-1]:],linestyle='dashed',color='b')
    ax1.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr][:mask[0]], -HillslopeALT_S60_loc10['temp_bottom-annual_avg'][yr][:mask[0]],linestyle='-',color='b',label=r'S$_{60}$ (b)')
    ax1.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr][mask[-1]:], -HillslopeALT_S60_loc10['temp_bottom-annual_avg'][yr][mask[-1]:],linestyle='-',color='b')

    #-------------------
    d_S90 = HillslopeALT_S90_loc10['temp_top-annual_avg'][yr] - HillslopeALT_S90_loc10['temp_bottom-annual_avg'][yr]
    mask = np.where(d_S90==0)[0]
    ax1.plot(HillslopeALT_S90_loc10['times-annual_avg'][yr], -HillslopeALT_S90_loc10['temp_top-annual_avg'][yr],linestyle='dashed',color='r',label=r'S$_{90}$ (t)')
    ax1.plot(HillslopeALT_S90_loc10['times-annual_avg'][yr], -HillslopeALT_S90_loc10['temp_bottom-annual_avg'][yr],linestyle='-',color='r',label=r'S$_{90}$ (b)')
    
ax1.set_title('Depth to base and top of the thaw layer (pond center)',fontsize='large', fontweight='normal')
ax1.set_xlabel('Time [Month of year]',fontsize='large', fontweight='normal')
ax1.set_ylabel('Depth [cm]',fontsize='large', fontweight='normal')
ax1.set_xlim(18.71,19.71)
ax1.set_xticklabels(['x', 'Oct', 'Dec', 'March', 'May', 'July'],fontsize=11, fontweight='normal')
ax1.set_yticklabels(np.linspace(90,0,10,dtype='i'),fontsize=11, fontweight='normal')
#ax1.set_ylim((-81, 1.5))
ax1.legend(loc='center left', fontsize=8,bbox_to_anchor=(.005,0.64, .1, .1),ncol=2)#, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
  
plt.tight_layout()
plt.savefig(outfile+'ALT-x%sm-5.pdf'%Loc, bbox_inches='tight', dpi=100)
plt.show()

[207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 262 263 264 265 266 267 268 269 270 271 272 273 274]
